In [ ]:
import os
import numpy as np

from sklearn import preprocessing
from sklearn import metrics

from keras.models import model_from_json

features = np.load('../data/features.npy')
labels = np.load('../data/labels.npy', allow_pickle=True)

test_indices = [np.load('../data/test_index1.npy'),
                np.load('../data/test_index2.npy'),
                np.load('../data/test_index3.npy'),
                np.load('../data/test_index4.npy'),
                np.load('../data/test_index5.npy')]

extra_benign = np.load('../data/extra_benign.npy')

models_folder = './Models_uniform'
results_folder = './Accuracies_uniform'
if not os.path.exists(results_folder): 
    os.mkdir(results_folder)
    print('Results folder created as', results_folder)

print('Index and data files loaded.')

In [ ]:
acc_path = os.path.join(results_folder,'overall_accuracy.npy')
conf_path = os.path.join(results_folder,'conf_matrix.npy')
cat_acc_path = os.path.join(results_folder,'category_accuracy.npy')

if not os.path.exists(acc_path) or not os.path.exists(conf_path) or not os.path.exists(cat_acc_path):

    overall_accuracy = [{}, {}]
    conf_matrix = [{}, {}]
    category_accuracy = [{}, {}]

    for folder in [f for f in os.listdir(models_folder) if not f.startswith('.')]:
        model_name = folder.split('_')[0]
        fold = int(folder.split('_')[1])-1
        model_path = os.path.join(models_folder, folder, 'model.json')
        weights_path = os.path.join(models_folder, folder, 'weights.h5')

        json_file = open(model_path, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        model.load_weights(weights_path)
        print('Loading model and weights for', folder + '...')

        mask = np.ones(len(labels), dtype=bool)
        mask[test_indices[fold],] = False
        mask[extra_benign,] = False; # commented out when testing the models trained with complete set (test_all and test_uniform are same.)
        x_train, x_test_all, x_test_uniform = features[mask], features[~mask], features[test_indices[fold],:]
        y_train, y_test_all, y_test_uniform = labels[mask], labels[~mask], labels[test_indices[fold]]

        minmax_scaler = preprocessing.MinMaxScaler()
        x_train = minmax_scaler.fit_transform(x_train)
        label_encoder = preprocessing.LabelEncoder()
        y_train = label_encoder.fit_transform(y_train)
        one_hot_encoder = preprocessing.OneHotEncoder(sparse=False)
        y_train = one_hot_encoder.fit_transform(y_train.reshape(-1,1))

        result_folder = os.path.join(results_folder,folder)        
        if not os.path.exists(result_folder): os.mkdir(result_folder)

        test_array = [(x_test_uniform, y_test_uniform), (x_test_all, y_test_all)]
        test_names = ['uniform', 'complete']
        print('Testing', folder, '...')

        for i in range(2): # range(1) when testing the models trained with complete set
            name = test_names[i]
            pred_path = os.path.join(result_folder, 'pred_' + name + '.npy' )
            y_test = label_encoder.transform(test_array[i][1])
            y_test = one_hot_encoder.transform(y_test.reshape(-1,1))
            y_test = np.argmax(y_test, axis=1)
            if os.path.exists(pred_path):
                print('Loading the existing predictions for model', model_name + ', fold', fold, 'and test', name + '...')
                pred = np.load(pred_path)
            else:
                print('Predicting the labels for model', model_name + ', fold', fold, 'and test', name + '...')
                x_test = minmax_scaler.transform(test_array[i][0])
                
                if not folder.startswith('mlp'):
                    x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

                pred = model.predict(x_test)
                pred = np.argmax(pred, axis=1)
                np.save(pred_path, pred)

            if model_name not in overall_accuracy[i]:
                overall_accuracy[i][model_name] = [[],[],[],[],[]]
            overall_accuracy[i][model_name][fold] = metrics.accuracy_score(y_test, pred)

            if model_name not in conf_matrix[i]:
                conf_matrix[i][model_name] = [[],[],[],[],[]]
            conf_m = metrics.confusion_matrix(y_test, pred)
            conf_matrix[i][model_name][fold] = conf_m

            if model_name not in category_accuracy[i]:
                category_accuracy[i][model_name] = [[],[],[],[],[]]
            category_accuracy[i][model_name][fold] = conf_m.diagonal()/conf_m.sum(axis=1)
    
        np.save(acc_path, overall_accuracy)
        np.save(conf_path, conf_matrix)
        np.save(cat_acc_path, category_accuracy)
        print('Results are saved.')
    print('All model tests are completed.')
else:
    overall_accuracy = np.load(acc_path, allow_pickle=True)
    conf_matrix = np.load(conf_path, allow_pickle=True)
    category_accuracy = np.load(cat_acc_path, allow_pickle=True)
    print('Existing results are loaded.')